Este código deberá correrse en una máquina remota que pueda soportarlo. Funcionó en una Ada 5000

No olvidar cargar los datos

In [ ]:
import torch
from evo2 import Evo2
import pandas as pd

evo2_model = Evo2('evo2_7b')

In [ ]:
brca1 = pd.read_excel("41586_2018_461_MOESM3_ESM.xlsx", header=2)
brca1 = brca1[[
    'chromosome', 'position (hg19)', 'reference', 'alt', 'function.score.mean', 'func.class',
]]

brca1.rename(columns={
    'chromosome': 'chrom',
    'position (hg19)': 'pos',
    'reference': 'ref',
    'alt': 'alt',
    'function.score.mean': 'score',
    'func.class': 'class',
}, inplace=True)
 
brca1['class'] = brca1['class'].replace(['FUNC', 'INT'], 'FUNC/INT')

In [ ]:
## testing
sequence = 'ACGTACGTACGTACGTACGT'
input_ids = torch.tensor(
    evo2_model.tokenizer.tokenize(sequence),
    dtype=torch.int,
).unsqueeze(0).to('cuda:0')

layer_name = 'blocks.28.mlp.l3'

outputs2, embeddings2 = evo2_model(input_ids, return_embeddings=True, layer_names=[layer_name])

print('Embeddings shape: ', embeddings2[layer_name].shape)

In [ ]:
WINDOW_SIZE = 128
complement_table = str.maketrans("ATCG", "TAGC")
def parse_sequences2(pos, ref, alt, chrn = "chr17"):
    match chrn:
        case "chr17":
            p = pos - 1 
            full_seq = chr17
        case "chr13":
            p = pos
            full_seq = chr13

    ref_seq_start = p - WINDOW_SIZE//2
    ref_seq_end = p + WINDOW_SIZE//2
    ref_seq = full_seq[ref_seq_start:ref_seq_end]
    snv_pos_in_ref = min(WINDOW_SIZE//2, p)
    var_seq = ref_seq[:snv_pos_in_ref] + alt + ref_seq[snv_pos_in_ref+1:]

    ref_seq_rev_comp = ref_seq.translate(complement_table)[::-1]
    var_seq_rev_comp = var_seq.translate(complement_table)[::-1]

    assert len(var_seq) == len(ref_seq)
    assert ref_seq[snv_pos_in_ref] == ref
    assert var_seq[snv_pos_in_ref] == alt

    return ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp

In [ ]:
from tqdm import tqdm
layer_name = 'blocks.28.mlp.l3'
idx_embd = {}

for idx, row in tqdm(brca1.iterrows(), total=len(brca1), desc="Procesando variantes de BRCA1"):
    ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp = parse_sequences2(row['pos'], row['ref'], row['alt'], "chr17")
    sequences_list = [ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp]
    input_ids = torch.tensor(
        [evo2_model.tokenizer.tokenize(s) for s in sequences_list],
        dtype=torch.int,
    ).to('cuda:0')
    _ , embeddings = evo2_model(input_ids, return_embeddings=True, layer_names=[layer_name])

    idx_embd[idx] = embeddings
    

In [ ]:
embeddings_avg = {idx: tensor['blocks.28.mlp.l3'].mean(dim=1) for idx, tensor in idx_embd.items()}
embeddings_f = {idx: embd.flatten() for idx, embd in embeddings_avg.items()}

In [ ]:
file_path = 'brca1_embeddings.pth'
torch.save(embeddings_f, file_path)